In [1]:
import json
import string
import random 
import nltk
import numpy as num
from nltk.stem import WordNetLemmatizer 
import tensorflow as tensorF
from tensorflow.keras import Sequential 
from tensorflow.keras.layers import Dense, Dropout
# nltk.download("punkt")
# nltk.download("wordnet")
# nltk.download('omw-1.4')

In [2]:
f=open('paraphrased.json')
data=json.load(f)

In [3]:
data.keys()

dict_keys(['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14'])

In [4]:
f=0
for intents in data:
  print(f)
  #print(data['0']['Patterns'])
  f+=1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


In [5]:
data['2']['Patterns']

['Please sir do it',
 'Sir i  my course registration  it showing gaming specialization but i have done registration  of only cybersecurity so pleasehelo',
 'change open elective ',
 "Respected sir/mam,\n\nI have requested for batch change it showed approved in the app but not yet updated in collpoll and I haven't received approved mail to  my outlook please help me out\n\nThank you.",
 'Sir, please do it.',
 'Sir i  my course registration  it showing gaming specialization but i have done registration  of only cybersecurity so pleasehelo',
 'Change elective ',
 "Respected sir/mam,\t\tI have requested for batch change it showed approved in the app but not yet updated in collpoll and I haven't received approved mail to  my outlook"]

In [6]:

#nltk.download('stopwords')
#nltk.download('wordnet')


import string
def text_process(text):

    stemmer = WordNetLemmatizer()
    nopunc = [char for char in text if char not in string.punctuation]
    nopunc = ''.join([i for i in nopunc if not i.isdigit()])
    nopunc =  [word.lower() for word in nopunc.split() if word not in stopwords.words('english')]
    return [stemmer.lemmatize(word) for word in nopunc]

In [7]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
stopwords.words('english').append(x for x in ['dear','student','students','Please','please','sir','dear'])

In [8]:
lm = WordNetLemmatizer()
ourClasses = []
newWords = []
documentX = []
documentY = []
for intent in data.keys():
    for pattern in data[intent]['Patterns']:
        #print(pattern)
        ourTokens = nltk.word_tokenize(pattern)
        newWords.extend(ourTokens)
        documentX.append(pattern)
        documentY.append(intent)

    if intent not in ourClasses:# add unexisting tags to their respective classes
        ourClasses.append(intent)
newWords = [lm.lemmatize(word.lower()) for word in newWords if word not in string.punctuation if word not in stopwords.words('english')] # set words to lowercase if not in punctuation
newWords = sorted(set(newWords))# sorting words
ourClasses = sorted(set(ourClasses))# sorting classes

In [27]:
trainingData = []
s="" 
outEmpty = [0] * len(ourClasses)
for idx, doc in enumerate(documentX):
    bagOfwords = []
    s=""
    k=doc.split()
    for x in k:
        x=x.lower()
        if x not in string.punctuation and x not in stopwords.words('english'):
            s+=x+" "
    text = lm.lemmatize(s)
    for word in newWords:
        bagOfwords.append(1) if word in text else bagOfwords.append(0)

    outputRow = list(outEmpty)
    outputRow[ourClasses.index(documentY[idx])] = 1
    trainingData.append([bagOfwords, outputRow])

random.shuffle(trainingData)
trainingData = num.array(trainingData, dtype=object)

x = num.array(list(trainingData[:, 0]))
y = num.array(list(trainingData[:, 1]))

In [13]:
for j in x:
    for k in j:
        if k!=0:
            print(k)

1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1


In [14]:
len(x[0])

1729

In [15]:
iShape = (len(x[0]),)
oShape = len(y[0])

ourNewModel = Sequential()
ourNewModel.add(Dense(128, input_shape=iShape, activation="relu"))

ourNewModel.add(Dense(64, activation="relu"))
ourNewModel.add(Dropout(0.3))
ourNewModel.add(Dense(oShape, activation = "sigmoid"))
ourNewModel.add(Dense(oShape,activation="softmax"))
md = tensorF.keras.optimizers.Adam()
ourNewModel.compile(loss='categorical_crossentropy',
              optimizer=md,
              metrics=["accuracy"])
print(ourNewModel.summary())
ourNewModel.fit(x, y, epochs=400, verbose=1)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 128)               221440    
                                                                 
 dense_1 (Dense)             (None, 64)                8256      
                                                                 
 dropout (Dropout)           (None, 64)                0         
                                                                 
 dense_2 (Dense)             (None, 15)                975       
                                                                 
 dense_3 (Dense)             (None, 15)                240       
                                                                 
Total params: 230,911
Trainable params: 230,911
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/400
37/37 [==========================

In [28]:
from sklearn.metrics import multilabel_confusion_matrix,ConfusionMatrixDisplay
y_test=ourNewModel.predict(x)
#multilabel_confusion_matrix(y,y_test)
y_test

37/37 [==============================] - 0s 2ms/step


array([[9.99294877e-01, 8.13372098e-05, 2.71347767e-06, ...,
        2.04928187e-06, 4.54758665e-05, 1.17057180e-05],
       [4.33053588e-04, 9.99371111e-01, 2.33660535e-09, ...,
        1.38700216e-08, 7.26183558e-10, 5.75226863e-07],
       [3.16835329e-04, 1.97772379e-03, 9.35834692e-07, ...,
        5.99320219e-06, 2.89442596e-07, 4.91188330e-05],
       ...,
       [4.40029107e-04, 9.99383569e-01, 2.01752326e-09, ...,
        1.29722615e-08, 6.54571786e-10, 5.30655143e-07],
       [1.10355950e-05, 8.87111470e-04, 4.56402449e-06, ...,
        1.99640723e-04, 1.89091451e-11, 7.09776723e-05],
       [1.33765798e-05, 8.95026198e-04, 5.41599866e-06, ...,
        2.13687526e-04, 2.20237491e-11, 8.25115712e-05]], dtype=float32)

In [71]:
l=[x for x in data.keys()]
l

['0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 '10',
 '11',
 '12',
 '13',
 '14']

In [33]:
def ourText(text):
  newtkns = nltk.word_tokenize(text)
  newtkns = [lm.lemmatize(word.lower()) for word in newtkns if word not in string.punctuation if word not in stopwords.words('english')]
  return newtkns

def wordBag(text, vocab):
  newtkns = ourText(text)
  bagOwords = [0] * len(vocab)
  for w in newtkns:
    for idx, word in enumerate(vocab):
      if word == w:
        bagOwords[idx] = 1
  return num.array(bagOwords)

def Pclass(text, vocab, labels):
  bagOwords = wordBag(text, vocab)
  ourResult = ourNewModel.predict(num.array([bagOwords]))[0]
  newThresh = 0.2
  yp = [[idx, res] for idx, res in enumerate(ourResult) if res > newThresh]

  yp.sort(key=lambda x: x[1], reverse=True)
  print(yp)
  newList = []
  for r in yp:
    newList.append(labels[r[0]])
  return newList

def getRes(firstlist, data):
  brr=" "
  tag = firstlist[0]
  listOfIntents = [x for x in data.keys()]
  for i in listOfIntents:
    if i == tag:
      #print("waaw")
      brr=random.choice(data[i]['Responses'])
      break
  return brr

In [34]:
i=0
while i<3:
    newMessage = input()
    intents = Pclass(newMessage, newWords, ourClasses)
    print(getRes(intents, data))
    i+=1

1/1 [==============================] - 0s 22ms/step
[[0, 0.4222733], [8, 0.29103026]]
Window is already closed, you can again try your luck in the next semester.
1/1 [==============================] - 0s 21ms/step
[[6, 0.92620385]]
Dear Student is it resolved now?
1/1 [==============================] - 0s 19ms/step
[[6, 0.92620385]]
I hope your query is resolved, I am closing this query.
you can raise a new ticket in case of another query.
Thanks,


In [36]:
testi=[]
for i in data.keys():
    j=data[i]['Patterns']
    k=data[i]['Responses']
    testi.append([j,k])

In [37]:
testi

[[['i am a 4th year(7th sem) btech CSE student and I want to drop application of AI course which is an open elective as I will be doing an internship for 6 months',
   'need batch changed from b1 to b2. conflict of interest with certain individuals, best way to avoid more drama is to seperate us.',
   "I wanted to change my batch from EB10 to EB02 but the option is mot available. My all the teammates for Capstone project is in EB02 . It's becoming difficult for me to manage as Capstone is the biggest project of the complete course.",
   'Sir, In batch change option I wanted to change my batch to EB-25 which was reflecting there but when I clicked on it. The page crashed and when I reload it, it was showing that my batch is changed to EB-21 but I wanted it to change to EB-25.',
   "Sir, I wanted to change my batch to EB-25 but due to some error it is changed to EB-21 sir please do change it to EB-25 coz I don't want to lose this opportunity. coz this is the last time we can change batch

In [38]:
import numpy as np
testi=np.array(testi)

C:\Users\devlo\AppData\Local\Temp\ipykernel_12904\2290509653.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  testi=np.array(testi)


In [61]:
j=0
w=0
wrng=[]
corec=[]
x=1
q=0
for i in testi:
    for k in i[0]:
            newMessage = k
            intents = Pclass(newMessage, newWords, ourClasses)
            f=getRes(intents, data)
            q+=1
            if f not in i[1]:
                w+=1
                if(x==7):
                        x+=1
                p="Category "+str(x)
                wrng.append([k,f,p])
                x+=1
            else:
                 j+=1
    corec.append([j,w+j])
    w=0
    j=0

1/1 [==============================] - 0s 22ms/step
[[0, 0.9990727]]
1/1 [==============================] - 0s 23ms/step
[[0, 0.9990938]]
1/1 [==============================] - 0s 18ms/step
[[0, 0.9984914]]
1/1 [==============================] - 0s 21ms/step
[[0, 0.99938285]]
1/1 [==============================] - 0s 22ms/step
[[6, 0.58110994], [0, 0.4131821]]
1/1 [==============================] - 0s 25ms/step
[[0, 0.50261027], [7, 0.44362858]]
1/1 [==============================] - 0s 25ms/step
[[0, 0.9993748]]
1/1 [==============================] - 0s 20ms/step
[[0, 0.99934345]]
1/1 [==============================] - 0s 24ms/step
[[0, 0.998716]]
1/1 [==============================] - 0s 20ms/step
[[0, 0.998887]]
1/1 [==============================] - 0s 24ms/step
[[0, 0.9892184]]
1/1 [==============================] - 0s 23ms/step
[[0, 0.99926776]]
1/1 [==============================] - 0s 25ms/step
[[0, 0.98334324]]
1/1 [==============================] - 0s 24ms/step
[[6, 0.5846768

In [62]:
from sklearn.metrics import r2_score
r2_score(y_test,y)

0.9523942830983111

In [63]:
import pandas as pd
wrng=pd.DataFrame(wrng,columns=["Input","Output Displayed","Category"])

In [64]:
wrng.to_csv('wrng.csv')

In [65]:
for x,i in enumerate(corec):
    print("Category {}".format(x),i[0]/i[1]*100)

Category 0 95.2127659574468
Category 1 93.5251798561151
Category 2 50.0
Category 3 86.11111111111111
Category 4 100.0
Category 5 62.5
Category 6 100.0
Category 7 85.71428571428571
Category 8 92.3076923076923
Category 9 97.22222222222221
Category 10 100.0
Category 11 90.27777777777779
Category 12 89.28571428571429
Category 13 91.66666666666666
Category 14 97.32142857142857


In [66]:
((q-len(wrng))/q)*100

94.27350427350427

In [59]:
#data['2']

In [58]:
#data.keys()